# <CENTER> Entrenamiento y trucos de redes neuronales profundas.</CENTER>
# <p><CENTER>Redes Neuronales convolucionales</CENTER>
# <p><CENTER>Modelo de detección de ciclones tropicales</CENTER>

###  Entrenando redes neuronales profundas

Entrenar una red neuronal profunda no es una tarea fácil.  Estos son los problemas con los que usted se podría encontrar:

1. La desaparición o explosión de gradientes: ocurre cuando los gradientes se hacen más y más pequeños o más y más grandes cuando se fluye en reversa durante la propagación en reversa durante el entrenamiento.  Estos problemas hacen muy dificil entrenar correctamente las últimas capas(layers).

2. Puede no tenersen suficientes datos de entrenamiento para redes profundas de muchas capas, o su etiquetado puede ser muy costoso.

3. Un modelo con millones de parámetros podría realizar sobre ajuste sobre el conjunto de datos de entrenamiento, especialmente si no hay suficientes datos o si tienen mucho ruido.

### La desaparición o explosión de gradientes:

El algoritmo de propagación en reversa (backpropagation algorihtm) trabaja al ir de las capas de salida a las capas de entrada, propagando el gradiente del error a lo largo del camino.  Una vez el algoritmo ha calculado los gradientes de la función objetivo con respecto a cada parámetro en la red, usa estos gradientes para actualizar cada parámetro con un paso de gradiente descendiente.

Pueden suceder dos cosas si van mal las cosas:

1. Los gradientes frecuentemente se vuelven más y más pequeños en la medida en que el algoritmo progresa haacia abajo de la red.  Como resultado de esto, el algoritmo del gradiente descendiente dejas los pesos de las conexiones virtualmente sin cambio alguno, y el entrenamiento nunca converge a una buena solución. A esto se le llama **la desaparición de los gradientes**

2. Lo opuesto puede suceder: los gradientes pueden hacerse más y más grandes hasta que los pesos de las capas obtienen valores demasiado grandes, haciendo que el algoritmo diverja.  A esto se le conoce como el problema de **la explosión de los gradientes**.  Problema especialmente común en redes neuronales recurrentes.

Este comportamiento fue visto hace tiempo, y fue una de las razones por las cúales las redes neuronales profundas fueron abandonadas al inicio de los 2000s.

No era claro que causaba que los gradientes fueran tan inestables cuando se entrenaba una red neuronal.  En 2010, Xavier Glorot y Yoshua Bengio, los aoutores encontraron un par de sospechosos que incluyen la combinación de la función de activación **Logística sigmoid** y la forma como se inicializan los pesos que era popular en ese tiempo, i.e. **distribución normal con media 0 y desviación estándar de 1**.

Se demostró que la varianza de las salidas de cada capa es mucho mayor que la varianza de sus entradas.  En la medidad en que avanza en la red, la varianza sigué creciendo hasta que la función de activación se satura en las últimas capas.   

Viendo la función de activación logistica, se puede ver que cuando las entradas son o pequeñas o grandes la función se satura a 0 o a 1, con derivada cercana a 0. 

### Glorot and He Initialization

En su artículo Glorot y Bengio proponen una forma de alivianar el problema de la inestabilidad de los gradientes. Hacen énfasis en que la señal debe fluir de forma apropiada en ambas direcciones:

1.  En la dirección adelante cuando hace la predicción 
2.  En la dirección reversa cuando esta propagando los gradientes.

No se desea que la señal muera, o que explote o se sature.

<p>
<center>
<img src="images/ann/sigmoid_saturation_plot.png" width="600">
</center>


Los autores argumentan que se necesita que la varianza de las salidas de cada capa sean iguales a la varianza de sus entradas y se necesita que los gradientes tengan igual varianza antes y después fluir a través de las capas en la dirección reversa. 

Glorot y Bengio propuso un buen compromiso que se ha probado funciona muy bien en la práctica:  La inicialización aleatoria de los pesos de cada capa debe hacerse como se describe a continuación:

$$
fan_{avg} = \frac{(fan_{in} + fan_{out})}{2}
$$

en donde $fan_{in}$ es el numero de entradas y $fan_{out}$ es el número de neuronas.  

Esta estrategia es llamada $\it{Xavier\;initialization\;o\;Glorot\;initialization}$.

Entonces:  Se usa una distribución Normal con media cero y varianza
$$
\sigma^2 = \frac{1}{fan_{avg}}
$$

Cuando se usa la **función de activación logística**.

<span style="color:blue"> El usar la inicialización de Glorot puede acelerar la velocidad del entrenamiento de forma considerable, y es uno de los trucos que permitió el éxito del entrenamiento profundo. </span>

### Al fin qué...todo normal?

$$
\begin{array}{|l|l|c|}
\hline
   \textbf{Inicialización} & \textbf{Funciones de activación} & \sigma^2(Normal) \\
\hline   
Glorot &  None, tanh, logistics, softmax & \frac{1}{fan_{avg}} \\
\hline   
He &  ReLU y\;variantes & \frac{2}{fan_{in}} \\
\hline   
LeCun & SELU & \frac{1}{fan_{in}} \\
\hline   
\end{array}
$$ 

Por  defecto, keras usa la inicialización Glorot con distribución uniforme.

Un ejemplo del código se escribe como sigue:

In [5]:
import tensorflow as tf
from tensorflow import keras
print(keras.__version__)
print(tf.__version__)       
    
keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal")

2.11.0
2.11.0


#### Funciones no saturantes

Tomado del paper: <span style="color:blue"> Understanding the difficulty of training deep feedforward neural networks, Xavier Glorot and Yoshua Bengio, DIRO, Université de Montréal, Montréal, Québec, Canada, 2010 Appearing in Proceedings of the 13 th International Conference
on Artificial Intelligence and Statistics.</span>

Uno de los problemas expuestos en el paper mencionado arriba, relacionado con la inestabilidad de los gradientes, eran en parte debido a una pobre selección de la función de activación. 

Se creía que la función de activación **sigmoide** que es la usada en la activación de las neuronas biológicas era la mejor opción.  Sin embargo, existen otras que se comportan mucho mejor en redes neuronales profundas.  En particular la función **ReLU**, por que no se satura para valores positivos y es rápida de calcular.

La **ReLU** también tiene sus problemas y eventualmente algunas neuronas mueren.


Tomado del paper: <span style="color:blue"> Self-Normalizing Neural Networks, Günter Klambauer
    ,Thomas Unterthiner y Andreas Mayr. Sepp Hochreiter.  LIT AI Lab & Institute of Bioinformatics, Johannes Kepler University Linz, A-4040 Linz, Austria, Sep. 7 2017.</span>
    
    
Se demostró, matemáticamente, que si se construye una red neuronal exclusivamente de un conjunto de capas densas, y si todas las capas ocultas usan la función de activación SELU (scaled Exponential Linear Unit), entonces la red se auto'normalizará: cada layer o capa van a tender a preservar la media de cero y una desviación estándar de 1 durante el entrenamiento.  Esto resuleve el problema de desaparación/explosión de los gradientes.

La función de activación **SELU** generálmente excede en desempeño a todas las otras, especialmente cuando se habla de redes neuronales profundas (DNN).
    
 

Condiciones para **usar la SELU**:
    
1. Las variables de entrada deben ser estandarizadas $\mu=0$ y $\sigma=1$.
2. Cada capa oculta debe inicializar los pesos usando LeCun normal.  En Keras estos ignifica usar kernel_initializer="lecun_normal" 
3. La red de la arquitectura debe ser secuencial.

Se garantiza la auto-normalización de la red si todas las capas son densas.  Sin embargo, se han encontrado beneficios en redes neuronales convolucionales.

In [6]:
keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal")

### Optimizadores
Una forma de acelerar el entrenamiento es a través del uso de optimizadores más rápidos que el gradiente descendiente:

Se recomienda:

$$
\begin{array}{|l|l|l|}
\hline
   \textbf{Clase} & \textbf{Vel. conver.} & \textbf{Cal. conver.} \\
\hline   
\textrm{SGD} &  * &  *** \\
\hline   
\textrm{Adagrad} &  *** &  * \\
\hline   
\textrm{RMSprop} &  *** & ** o *** \\
\hline   
\textrm{Adam} &  *** &  ** o *** \\
\hline   
\textrm{NAdam} &  *** &  ** o *** \\
\hline   
\end{array}
$$ 



### Fórmula para actualizar  los parámetros usando gradiente descendente

**Gradient Descent:**

Gradient descent is based on the observation that if the multi-variable function $F(\mathbf{x})$ is defined and differentiable in a neighborhood of a point  $\mathbf{a}$ , then $F(\mathbf{x})$ decreases fastest if one goes from $\mathbf{a}$ in the direction of the negative gradient of $F$ at ${\displaystyle \mathbf{a} ,-\nabla F(\mathbf{a})}$. It follows that,

$$
\mathbf{a}_{n+1} = \mathbf{a}_n - \gamma \nabla F(\mathbf{a}_{n})
$$

for a small enough step size or learning rate ${\displaystyle \gamma \in \mathbb {R} _{+}}$, then ${\displaystyle F(\mathbf {a_{n}} )\geq F(\mathbf {a_{n+1}} )}$.

Fuente: **wikipedia**

### Tasas de aprendizaje

<p>
<center>
<img src="figuras/learning-rate.png" width="600">
</center>


<center>
<h1>
    REDES NEURONALES CONVOLUCIONALES
</h1>
</center>

CNNs (Convolutional Neural networks) are inspired by the organization of animal visual cortex in biological proceses.

The overlapping sub-regions of the visual field, called receptive fields, are obtained through the collection of small neurons across multiple layers in a CNN.

This overlapping mechanism  allows CNNs to tolerate the translation of an input image.

### comments on the RELU  activation function used in the Tropical cyclone pattern classification problem

1. The Rectified Linear Unit Function $RELU(Z) = MAX(0,z)$ is continuous but not differentiable at $z=0$
2. The derivative is zero  for $z < 0$. Might cause gradient dissapearing
3. The  slope (derivative) chaanges abruptly and can cause gradient descent to bounce around.
4. It is fast and  it works well most of the time.  Widely used.